In [1]:
import os
import pandas as pd
import numpy as np
import pyautogui
import urllib.parse
import time
import string
import pyperclip
import quopri
from bs4 import BeautifulSoup
from email import policy
from email.parser import BytesParser
import psycopg2
import requests
import json
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize

# 1.Data Collection

(1). Download LinkedIn Search Results Webpages

In [2]:
# Create an automatic process to download the job search pages in Linkedin
def auto_download_search(url,file_name):
    pyautogui.hotkey('ctrl', '2') # switch Google Chrome tabs
    time.sleep(1)
    
    pyautogui.click(207,55) # click the browser's address bar.
    time.sleep(0.5)
    
    pyautogui.hotkey('ctrl', 'a') # select all content in the address bar.
    pyperclip.copy(url) # copy the url
    time.sleep(0.5)
    
    pyautogui.hotkey('ctrl', 'v') # paste the url
    time.sleep(0.5)
    
    pyautogui.hotkey('enter') # access the url
    time.sleep(10)
    
    pyautogui.click(882,1023)
    pyautogui.mouseDown() # scroll down the page to look through all job posts
    time.sleep(10)
    pyautogui.mouseUp()# release the left mouse button
    
    pyautogui.hotkey('ctrl', 's') # save the page to local path
    time.sleep(2)
    pyperclip.copy(file_name) # copy the file name
    pyautogui.hotkey('ctrl', 'v') # paste the file name
    time.sleep(2)
    pyautogui.hotkey('enter') # save the page with given name

In [3]:
# create a process to look through all the pages of search results
def search_page_download(num = 0):
    for i in range(0,1000,25):
        url = f"https://www.linkedin.com/jobs/search/?currentJobId=3733476715&f_SB2=1&f_TPR=r2592000&geoId=103644278&keywords=data%20scientist&location=United%20States&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=R&spellCorrectionEnabled=true&start={i}"
        file_name = f"data_science_job_{num + int(i/25)}.mhtml"
        auto_download_search(url, file_name)

In [4]:
# download all pages to collect 1000 job posts related to data science
search_page_download(40)

(2). Extract Individual Job Post URLs from LinkedIn Search Results

In [5]:
# extract urls of job posts from all the pages
def job_url(file_path):
    urls = []
    # Open the MHTML file in binary mode and parse it
    with open(file_path, 'rb') as file:
        msg = BytesParser(policy=policy.default).parse(file)

    # Decode the HTML part correctly
    html_part = None
    for part in msg.walk():
        content_type = part.get_content_type()
        if content_type == 'text/html':
            html_part = part.get_payload(decode=True)
            break
    
    # parse the HTML
    charset = 'utf-8'
    decoded_html = html_part.decode(charset)
    soup = BeautifulSoup(decoded_html, 'html.parser')
    jobs = soup.find(class_ = "scaffold-layout__list-container")
    job_info = jobs.find_all(class_ = "disabled ember-view job-card-container__link job-card-list__title")
    # extract the url for each job post
    for i in job_info:
        url = i['href']
        urls.append(url)
    return urls

In [6]:
job_urls = {}
i = 0
path = "D:/Study Abroad/course/DSCI441/project/webpages"

# get the urls of all job posts from the local job pages
for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        urls = job_url(file_path)
        for url in urls:
            job_urls[i] = {}
            job_urls[i]["url"] = url
            job_urls[i]["status"] = 0
            i += 1

(3). Download Job Post Details Webpages

In [7]:
# manually login the linkedin website
driver = webdriver.Chrome()
driver.get("https://www.linkedin.com/home")

# wait enough time to log in manually
time.sleep(20)  

# save the cookies to a file after login
pickle.dump(driver.get_cookies(), open("D:/Study Abroad/course/DSCI441/project/cookies.pkl", "wb"))

driver.quit()

In [8]:
# use selenium webdriver to iterate all the job pages
# set the options
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# load the previously saved cookies
cookies = pickle.load(open("D:/Study Abroad/course/DSCI441/project/cookies.pkl", "rb"))

# iterate all job post pages and download them to local path
for i in range(0,1000):
    index = i
    value = job_urls[index]
    
    # open a new driver after iterating the job post pages for 20 times in order to save memory
    if index % 20 == 0:
        driver = webdriver.Chrome(options=options)
        driver.get("https://www.linkedin.com/home")
        time.sleep(4)
        
        # load the cookies to the driver
        for cookie in cookies:
            driver.add_cookie(cookie)
        time.sleep(1)
    
    # get the url for job post
    url = value["url"]
    driver.get(url)
    
    time.sleep(5)
    
    # use the aria-label to locate the button
    button_aria_label = "Click to see more description"
    try:
        # Wait up to 10 seconds for the button to be clickable
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//button[@aria-label='{button_aria_label}']")))
        # click the button "Show More" to show the entire job description
        button.click()
        
        time.sleep(2) # wait up for all information displayed
        
    except Exception as e:
        break # if failed, jump to the next iteration
    
    pyautogui.hotkey('ctrl', 's') # save the page to local path
    time.sleep(2)
    
    file_name = "data_scientist_job_post_" + str(index)
    pyperclip.copy(file_name) # copy the file name
    
    pyautogui.hotkey('ctrl', 'v') # paste the file name
    time.sleep(2)
    
    if index % 20 == 0:
        pyautogui.hotkey('tab')  # select the save type
        time.sleep(1)

        pyautogui.hotkey('down')
        time.sleep(1)

        pyautogui.hotkey('up')  # save file to mhtml
        time.sleep(1)

        pyautogui.hotkey('enter') # confirm the file tyep
        time.sleep(1)
    
    pyautogui.hotkey('enter') # save
    time.sleep(np.random.randint(1,4)) # wait for random time
    
    if index % 20 == 19:
        driver.quit()

(4). Extract Key Information from Job Posting Webpages

In [9]:
# categorize all job_type, remote, and experience
JOB_TYPE = ["Full-time","Part-time","Contract","Temporary","Internship","Other"]
REMOTE_TYPE = ["On-site","Remote","Hybrid"]
EXPERIENCE_TYPE = ["Internship","Entry level","Associate","Mid-Senior level","Director","Executive"]

In [10]:
# define functions to extract job_type, remote, and experience information
def extract_info(info, options):
    for i in options:
        if i in info:
            return i
    return None

In [11]:
# create variables to store the details of all the jobs
indice = []
titles = []
companies = []
locations = []
job_types = []
remotes = []
experiences = []
salaries = []
company_sizes = []
fields = []
job_descriptions = []
skills_posted = []
skills_associated = []

In [5]:
# get the urls of all job posts from the local job pages
path = "D:/Study Abroad/course/DSCI441/project/jobposts"

for root, dirs, files in os.walk(path):
    for file in files:
        file_path = os.path.join(root, file)
        # Open the MHTML file in binary mode and parse it
        with open(file_path, 'rb') as file:
            msg = BytesParser(policy=policy.default).parse(file)

        # Decode the HTML part correctly
        html_part = None
        for part in msg.walk():
            content_type = part.get_content_type()
            if content_type == 'text/html':
                html_part = part.get_payload(decode=True)
                break

        # parse the HTML
        charset = 'utf-8'
        decoded_html = html_part.decode(charset)
        soup = BeautifulSoup(decoded_html, 'html.parser')
        
        # extract the index
        index = file_path.replace(".mhtml","").split("_")[-1]
        indice.append(index)
        
        # extract title
        title = soup.find(class_="t-24 t-bold job-details-jobs-unified-top-card__job-title").contents[0].strip()
        titles.append(title)
        
        # extract company and location
        company_location = soup.find(class_="job-details-jobs-unified-top-card__primary-description-without-tagline mb2").text.strip()
        company = company_location.split('·')[0].strip()
        location = company_location.split('·')[1].strip()
        companies.append(company)
        locations.append(location)
        
        # extract job_type, remote, experience, and salary
        salary_jobtype_Remote_Experience = soup.find(class_="job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight").text
        job_type = extract_info(salary_jobtype_Remote_Experience,JOB_TYPE)
        remote = extract_info(salary_jobtype_Remote_Experience,REMOTE_TYPE)
        experience = extract_info(salary_jobtype_Remote_Experience,EXPERIENCE_TYPE)
        salary = salary_jobtype_Remote_Experience.strip().split('\n')[0]
        job_types.append(job_type)
        remotes.append(remote)
        experiences.append(experience)
        salaries.append(salary)
        
        # extract company size and field
        companysize_field = soup.find_all(class_ = "job-details-jobs-unified-top-card__job-insight")[1].text.split('·')
        company_size = companysize_field[0].strip()
        company_sizes.append(company_size)
        if len(companysize_field)>1: 
            field = companysize_field[1].strip() 
        else:
            field = None
        fields.append(field)
        
        # extract the job description
        job_description = soup.find(class_ = "jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch").text.strip()
        job_descriptions.append(job_description)

        # extract the associated skills and skills added by job posters
        skill_posted = [item.text for item in soup.find_all(class_="pt5")[1].find_all("a",class_="app-aware-link job-details-how-you-match__skills-item-subtitle t-14 overflow-hidden")]
        skill_associated = [item.text for item in soup.find_all(class_="pt5")[1].find_all("a",class_="app-aware-link job-details-how-you-match__skills-section-descriptive-skill t-14")]
        skills_posted.append(skill_posted)
        skills_associated.append(skill_associated)

In [6]:
df = pd.DataFrame((titles,companies,locations,job_types,remotes,experiences,salaries,company_sizes,fields,job_descriptions,skills_posted,skills_associated,indice)).T
df.columns=["Job_Title","Company_Name","Location","Job_Type","Remote","Experience_Level","Salary","Company_Size","Field","Job_Description","Posted_Skills","Associated_Skills","Files"]

(5). Store the information into postgres database

In [12]:
def insert_database(df,connection):
    cursor = connection.cursor()
    data = df.drop('Files',axis = 1).copy()
    data = data.fillna("Unknown")
    # insert into database
    try:
        for fields in data.values:
            insert_query = "INSERT INTO t_ds_job_info"
            insert_columns = '("' + '","'.join(data.columns) + '")'
            insert_values = "(" 
            for field in fields:
                insert_values += "'" + str(field).replace("'","").replace('"','') + "',"
            insert_values = insert_values[:-1]
            insert_values +=  ")"
            insert_query = insert_query + insert_columns + " VALUES " + insert_values
            cursor.execute(insert_query)
            connection.commit()
    except:
        print(insert_query)

    # Close the connection
    cursor.close()
    connection.close()
    
# a function to extract data from database
def query(SQL,connection):
    df = pd.read_sql(SQL,connection)
    connection.close()
    
    return df

In [8]:
# define the connection
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
# save the information
insert_database(df,connection)

In [13]:
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
# save the information
SQL = 'select distinct "Job_Title","Company_Name" ,"Location","Job_Type","Remote","Experience_Level","Salary","Company_Size" ,"Field" ,"Job_Description" ,"Posted_Skills","Associated_Skills" from t_ds_job_info'
df = query(SQL,connection)

C:\Users\Xiang Gao\AppData\Local\Temp\ipykernel_7004\3009979463.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL,connection)


In [14]:
df

,Job_Title,Company_Name,Location,Job_Type,Remote,Experience_Level,Salary,Company_Size,Field,Job_Description,Posted_Skills,Associated_Skills
0,"Data Scientist II, Amazon Business Ops Analytics",Amazon,"Arlington, VA",Full-time,Unknown,Mid-Senior level,"$111,600/yr - $212,800/yr","10,001+ employees",Software Development,About the job\n \n \nDescriptionWe ...,[],"[Machine Learning · Extract, Transform, Load (..."
1,Data Scientist,CACI International Inc,"Denver, CO",Full-time,Unknown,Mid-Senior level,"$82,100/yr - $172,400/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nJob Category: ...,[],[Logistic Regression · Computer Vision · Machi...
2,Data Scientist with Security Clearance,ClearanceJobs,"Arlington, VA",Full-time,Hybrid,Mid-Senior level,"$155,000/yr",11-50 employees,Defense and Space Manufacturing,About the job\n \n \nAward Winning ...,[],[Data Mining · HTML · Data Analytics · SQL · P...
3,"Sr Data Scientist, ProServe GenAI - Open Reqs",Amazon Web Services (AWS),"Santa Clara, CA",Full-time,Unknown,Mid-Senior level,"$127,300/yr - $247,600/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nDescriptionAre...,[],[Machine Learning · Artificial Intelligence (A...
4,Data Scientist,Insight Global,"Seattle, WA",Contract,On-site,Mid-Senior level,$70/hr - $80/hr,"1,001-5,000 employees",Staffing and Recruiting,About the job\n \n \nSr. Data Scien...,[],[Python (Programming Language) · Applied Machi...
...,...,...,...,...,...,...,...,...,...,...,...,...
616,Mission Solution SAFe Shared Services- Data Sc...,CACI International Inc,"Aurora, CO",Full-time,On-site,Mid-Senior level,"$104,200/yr - $229,200/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nJob Category: ...,[],[R (Programming Language) · Big Data Analytics...
617,Data Scientist,Booz Allen Hamilton,"Huntsville, AL",Full-time,Unknown,Unknown,"$73,000/yr - $166,000/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nJob Number: R0...,[],[Data Mining · Analytics · Data Analytics · Pa...
618,"Data Scientist, Analytics - Tiktok",TikTok,"San Jose, CA",Full-time,Unknown,Unknown,"$144,000/yr - $240,000/yr","10,001+ employees",Entertainment Providers,About the job\n \n \nResponsibiliti...,[],[Data Visualization · Natural Language Process...
619,"Data Scientist, Mid",Booz Allen Hamilton,"Smith, IN",Full-time,Unknown,Unknown,"$73,000/yr - $166,000/yr","10,001+ employees",IT Services and IT Consulting,About the job\n \n \nJob Number: R0...,[],[Python (Programming Language) · Analytics · D...


# 2.Data Processing

#### (1). Reformat numerical and categorical features

In [15]:
# define function to extract state from location
def location_to_state(value):
    area_state_dict = {"United States":"Unknown",
                 "Los Angeles Metropolitan Area": "CA",
                 "Greater Seattle Area": "WA",
                 "New York City Metropolitan Area": "NY",
                 "Greater Minneapolis-St. Paul Area": "MN",
                 "San Francisco Bay Area": "CA",
                 "Miami-Fort Lauderdale Area": "FL",
                 "Atlanta Metropolitan Area": "GA",
                 "Washington DC-Baltimore Area": "DC",
                 "Charlotte Metro": "NC",
                 "Greater Phoenix Area": "AZ",
                 "Dallas-Fort Worth Metroplex": "TX",
                 "Greater Richmond Region": "VA" ,
                 "San Diego Metropolitan Area": "CA",
                 "Greater Boston": "MA",
                 "Oregon Metropolitan Area":"OR",
                 'Alabama': 'AL',
                 'Alaska': 'AK',
                 'Arizona': 'AZ',
                 'Arkansas': 'AR',
                 'California': 'CA',
                 'Colorado': 'CO',
                 'Connecticut': 'CT',
                 'Delaware': 'DE',
                 'Florida': 'FL',
                 'Georgia': 'GA',
                 'Hawaii': 'HI',
                 'Idaho': 'ID',
                 'Illinois': 'IL',
                 'Indiana': 'IN',
                 'Iowa': 'IA',
                 'Kansas': 'KS',
                 'Kentucky': 'KY',
                 'Louisiana': 'LA',
                 'Maine': 'ME',
                 'Maryland': 'MD',
                 'Massachusetts': 'MA',
                 'Michigan': 'MI',
                 'Minnesota': 'MN',
                 'Mississippi': 'MS',
                 'Missouri': 'MO',
                 'Montana': 'MT',
                 'Nebraska': 'NE',
                 'Nevada': 'NV',
                 'New Hampshire': 'NH',
                 'New Jersey': 'NJ',
                 'New Mexico': 'NM',
                 'New York': 'NY',
                 'North Carolina': 'NC',
                 'North Dakota': 'ND',
                 'Ohio': 'OH',
                 'Oklahoma': 'OK',
                 'Oregon': 'OR',
                 'Pennsylvania': 'PA',
                 'Rhode Island': 'RI',
                 'South Carolina': 'SC',
                 'South Dakota': 'SD',
                 'Tennessee': 'TN',
                 'Texas': 'TX',
                 'Utah': 'UT',
                 'Vermont': 'VT',
                 'Virginia': 'VA',
                 'Washington': 'WA',
                 'West Virginia': 'WV',
                 'Wisconsin': 'WI',
                 'Wyoming': 'WY'}
    
    if "," not in value:
        state = area_state_dict.get(value.strip(),"Unknown")
    elif len(value.split(",")[1].strip())>2 and value.split(",")[1].strip()!="United States":
        state = area_state_dict.get(value.split(',')[1].strip())
    elif len(value.split(",")[1].strip())>2 and value.split(",")[1].strip()=="United States":
        state = area_state_dict.get(value.split(',')[0].strip(),"Unknown")
    else:
        state = value.split(',')[1].strip()
    
    return state

# define function to extract the salary range
def salary_to_annual_salary_range(value):
    if "-" not in value and "/hr" in value:
        value = value.replace("/hr","")
        value = value.replace("$","") 
        value = value.replace(",","")
        upper = float(value) * 40 * 52
        lower = float(value) * 40 * 52
        
    elif "-" not in value and "/yr" in value:
        value = value.replace("/yr","")
        value = value.replace("$","") 
        value = value.replace(",","")
        value = value.replace("Up to ","")
        upper = float(value)
        lower = float(value)
    
    elif "-" in value and "/hr" in value:
        value = value.replace("/hr","")
        value = value.replace("$","") 
        value = value.replace(",","")
        upper = float(value.split("-")[0].strip())  * 40 * 52
        lower = float(value.split("-")[1].strip())  * 40 * 52
    
    elif "-" in value and "/month" in value:
        value = value.replace("/month","")
        value = value.replace("$","") 
        value = value.replace(",","")
        upper = float(value.split("-")[0].strip()) * 12
        lower = float(value.split("-")[1].strip()) * 12

    elif "-" in value and "/yr" in value:
        value = value.replace("/yr","")
        value = value.replace("$","") 
        value = value.replace(",","")
        upper = float(value.split("-")[0].strip())
        lower = float(value.split("-")[1].split("+")[0].strip())
    return upper, lower

# define function to quantize the avg company size 
def avg_company_size(value):
    dict_company_size = {'10,001+ employees':  20000,
                         '5,001-10,000 employees': 7500,
                         '1,001-5,000 employees': 3000,
                         '501-1,000 employees': 750,
                         '201-500 employees': 350,
                         '51-200 employees': 125,
                         '11-50 employees' : 30,
                         '1-10 employees' : 5                       
                        }
    company_size = dict_company_size.get(value,0)
    return company_size

In [16]:
# extract the state information
# drop the city and area information since it is too detailed and may cause overfitting after one-hot encoding
df["State"] = df["Location"].apply(location_to_state)

# extract the range of salary
df["Salary_Lower"] = df["Salary"].apply(salary_to_annual_salary_range).apply(lambda x:x[0])
df["Salary_Upper"] = df["Salary"].apply(salary_to_annual_salary_range).apply(lambda x:x[1])

# quantize the Company_Size using the avg value
df['Company_Size'] = df['Company_Size'].apply(avg_company_size)

#### (2). Handle text features

In [17]:
# define function to identify the english words included in the job title
def words_identification_title(value):
    punctuations = string.punctuation
    # Replace each punctuation mark with a space
    for p in punctuations:
        value = value.replace(p, " ")
    value = value.split(" ")
    value = [word.strip() for word in value]
    
    return value

# define function to identify the english words included in the job description
def words_identification_description(value):
    english_words = set(words.words())
    tokens = word_tokenize(value)
    identified_words = [token.strip() for token in tokens if token.lower() in english_words]
    return identified_words

# define function to identify the skills in the Posted_Skills
def posted_skillset(value):
    value = value.replace('[','')
    value = value.replace(']','')
    value = value.replace(', and',',')
    skillset = value.split(",")
    skillset = [skill.strip() for skill in skillset]
    if skillset == ['']:
        skillset = []
        
    return skillset

# define function to identify the skills in the Associated_Skills
def associated_skillset(value):
    value = value.replace('[','')
    value = value.replace(']','')
    skillset = value.split('·')
    skillset = [skill.strip() for skill in skillset]
    if skillset == ['']:
        skillset = []
    return skillset

In [18]:
nltk.download('words')
nltk.download('punkt')

# extract words from job title
df['Job_Title_Words'] = df['Job_Title'].apply(words_identification_title)

# extract words from job description
df['Job_Description_Words'] = df['Job_Description'].apply(words_identification_description)

# extract posted skills
df['Posted_Skills'] = df['Posted_Skills'].apply(posted_skillset)

# extract associated skills
df['Associated_Skills'] = df['Associated_Skills'].apply(associated_skillset)

[nltk_data] Downloading package words to C:\Users\Xiang
[nltk_data]     Gao\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Xiang
[nltk_data]     Gao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
df = df[['Job_Title_Words','Company_Name','State','Job_Type','Remote','Experience_Level','Company_Size','Field', 'Job_Description_Words','Posted_Skills','Associated_Skills' ,'Salary_Lower','Salary_Upper']]
df

,Job_Title_Words,Company_Name,State,Job_Type,Remote,Experience_Level,Company_Size,Field,Job_Description_Words,Posted_Skills,Associated_Skills,Salary_Lower,Salary_Upper
0,"[Data, Scientist, II, , Amazon, Business, Ops,...",Amazon,VA,Full-time,Unknown,Mid-Senior level,20000,Software Development,"[About, the, job, are, seeking, a, Data, Scien...",[],"[Machine Learning, Extract, Transform, Load (E...",111600.0,212800.0
1,"[Data, Scientist]",CACI International Inc,CO,Full-time,Unknown,Mid-Senior level,20000,IT Services and IT Consulting,"[About, the, job, Job, Category, Type, Full, C...",[],"[Logistic Regression, Computer Vision, Machine...",82100.0,172400.0
2,"[Data, Scientist, with, Security, Clearance]",ClearanceJobs,VA,Full-time,Hybrid,Mid-Senior level,30,Defense and Space Manufacturing,"[About, the, job, Award, Winning, Business, An...",[],"[Data Mining, HTML, Data Analytics, SQL, Postg...",155000.0,155000.0
3,"[Sr, Data, Scientist, , ProServe, GenAI, , , O...",Amazon Web Services (AWS),CA,Full-time,Unknown,Mid-Senior level,20000,IT Services and IT Consulting,"[About, the, job, you, looking, to, work, at, ...",[],"[Machine Learning, Artificial Intelligence (AI...",127300.0,247600.0
4,"[Data, Scientist]",Insight Global,WA,Contract,On-site,Mid-Senior level,3000,Staffing and Recruiting,"[About, the, job, Data, Scientist, Hardware, a...",[],"[Python (Programming Language), Applied Machin...",145600.0,166400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,"[Mission, Solution, SAFe, Shared, Services, , ...",CACI International Inc,CO,Full-time,On-site,Mid-Senior level,20000,IT Services and IT Consulting,"[About, the, job, Job, Category, Type, Full, C...",[],"[R (Programming Language), Big Data Analytics,...",104200.0,229200.0
617,"[Data, Scientist]",Booz Allen Hamilton,AL,Full-time,Unknown,Unknown,20000,IT Services and IT Consulting,"[About, the, job, Job, Number, Opportunity, As...",[],"[Data Mining, Analytics, Data Analytics, Panda...",73000.0,166000.0
618,"[Data, Scientist, , Analytics, , , Tiktok]",TikTok,CA,Full-time,Unknown,Unknown,20000,Entertainment Providers,"[About, the, job, is, the, leading, destinatio...",[],"[Data Visualization, Natural Language Processi...",144000.0,240000.0
619,"[Data, Scientist, , Mid]",Booz Allen Hamilton,IN,Full-time,Unknown,Unknown,20000,IT Services and IT Consulting,"[About, the, job, Job, Number, Scientist, Oppo...",[],"[Python (Programming Language), Analytics, Dat...",73000.0,166000.0
